# Automated Response
- **Objective:** To generate alerts when certain fault conditions occur

In [1]:
# Import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  # Split data for training and testing
from sklearn.preprocessing import LabelEncoder  # Convert text to numbers
from sklearn.tree import DecisionTreeClassifier  # Machine learning model
from sklearn.metrics import classification_report, confusion_matrix  # Check how well the model works

In [2]:
# Read data file
df = pd.read_csv('lighting_data_cleaned.csv')

# Display data
df

,light_id,location_name,fault_type,timestamp,severity_level,fault_status,maintenance_cost,year,month,day,day_of_week,hour
0,L0313,Jurong West,Power-Related,2020-01-07 09:05:35,Low,Closed,297.45,2020,1,7,Tuesday,9
1,L0385,Pasir Ris,Environmental,2020-01-07 10:28:12,Medium,In Progress,172.61,2020,1,7,Tuesday,10
2,L0158,Canberra,Communication,2020-01-07 13:09:56,Medium,Detected,381.23,2020,1,7,Tuesday,13
3,L0554,Jurong West,Communication,2020-01-08 01:55:51,Medium,In Progress,284.30,2020,1,8,Wednesday,1
4,L0755,Bishan,Communication,2020-01-11 10:49:07,Critical,In Progress,76.52,2020,1,11,Saturday,10
...,...,...,...,...,...,...,...,...,...,...,...,...
163,L0768,Chinatown,Cybersecurity,2020-12-24 21:14:30,Critical,Detected,156.92,2020,12,24,Thursday,21
164,L0716,Pasir Ris,Control System,2020-12-26 03:30:47,High,Closed,262.23,2020,12,26,Saturday,3
165,L0085,Toa Payoh,Communication,2020-12-27 02:25:18,Medium,Closed,377.93,2020,12,27,Sunday,2
166,L0525,Pasir Ris,Control System,2020-12-28 00:39:48,Low,In Progress,191.01,2020,12,28,Monday,0


In [3]:
# Filter faults with severity High or Critical
alert_faults = df[df['severity_level'].isin(['High', 'Critical'])]

# Filter df to include only rows where:
power_alerts = df[
    (df['fault_type'] == 'Power-related') &  # Condition 1: fault_type must be 'Power-related'
    (df['severity_level'].isin(['Medium', 'High', 'Critical']))  # Condition 2: severity_level must be one of these values
]

all_alerts = (
    pd.concat([alert_faults, power_alerts])  # Combine the two filtered dfs 'alert_faults' and 'power_alerts' into one df
    .drop_duplicates()                      # Remove duplicates
    .reset_index(drop=True)                 # Reset index starting at 0
)

# Print the total number of alert records in the 'all_alerts' df
print(f"Total alerts generated: {len(all_alerts)}")

# Alert display
for idx, row in all_alerts.iterrows():
    print(f"ALERT: Fault '{row['fault_type']}' at '{row['location_name']}' with severity '{row['severity_level']}' on {row['timestamp']}")

Total alerts generated: 68
ALERT: Fault 'Communication' at 'Bishan' with severity 'Critical' on 2020-01-11 10:49:07
ALERT: Fault 'Cybersecurity' at 'Toa Payoh' with severity 'Critical' on 2020-01-15 17:19:49
ALERT: Fault 'Control System' at 'Jurong West' with severity 'High' on 2020-01-16 13:53:28
ALERT: Fault 'Power-Related' at 'Chinatown' with severity 'Critical' on 2020-01-17 03:41:25
ALERT: Fault 'Control System' at 'Chinatown' with severity 'High' on 2020-01-17 22:03:22
ALERT: Fault 'Cybersecurity' at 'City Hall' with severity 'High' on 2020-01-25 04:05:58
ALERT: Fault 'Environmental' at 'Orchard Road' with severity 'Critical' on 2020-01-25 11:30:17
ALERT: Fault 'Environmental' at 'Orchard Road' with severity 'Critical' on 2020-01-31 02:51:06
ALERT: Fault 'Communication' at 'Jurong West' with severity 'High' on 2020-02-02 22:13:59
ALERT: Fault 'Cybersecurity' at 'Bishan' with severity 'Critical' on 2020-02-06 04:23:36
ALERT: Fault 'Environmental' at 'Outram Park' with severity 'Hi

In [4]:
# Create a new DataFrame with only selected columns from all_alerts
styled_alerts = all_alerts[['fault_type', 'location_name', 'severity_level', 'timestamp']].copy()

# Define severity levels colours
severity_colors = {
    "Critical": "background-color: #ff4d4d; color: white; font-weight: bold;", # Red
    "High": "background-color: #ffa500; color: white; font-weight: bold;", # Orange
    "Medium": "background-color: #ffeb3b; color: black;" # Yellow
}

# Return colour for the given severity level
def highlight_severity(val):
    return severity_colors.get(val, "")

# Apply color styling to the severity_level column and set table styles
styled_alerts.style.applymap(highlight_severity, subset=['severity_level']) \
    .set_table_styles([
        {'selector': 'thead th', 'props': [('background-color', '#333'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('border', '1px solid #ccc'), ('padding', '6px')]},
    ]) \
    .set_properties(**{'text-align': 'center'})

/tmp/ipython-input-1786090863.py:16: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_alerts.style.applymap(highlight_severity, subset=['severity_level']) \


,fault_type,location_name,severity_level,timestamp
0,Communication,Bishan,Critical,2020-01-11 10:49:07
1,Cybersecurity,Toa Payoh,Critical,2020-01-15 17:19:49
2,Control System,Jurong West,High,2020-01-16 13:53:28
3,Power-Related,Chinatown,Critical,2020-01-17 03:41:25
4,Control System,Chinatown,High,2020-01-17 22:03:22
5,Cybersecurity,City Hall,High,2020-01-25 04:05:58
6,Environmental,Orchard Road,Critical,2020-01-25 11:30:17
7,Environmental,Orchard Road,Critical,2020-01-31 02:51:06
8,Communication,Jurong West,High,2020-02-02 22:13:59
9,Cybersecurity,Bishan,Critical,2020-02-06 04:23:36
